In [31]:
from scipy.sparse import csc_array
from scipy.sparse.linalg import bicgstab
from scipy.sparse.linalg import cg
from scipy.sparse.linalg import gmres
import numpy as np
import matplotlib.pyplot as plt
pi = np.pi

from copy import deepcopy

In [32]:
def DatosMat(path):
    data = [[],[],[],[]]
    with open(path) as fobj:
        for i, line in enumerate(fobj):
           # data.append([])
            if line.rsplit() != []:
                row = line.rsplit()   
                for j in range(len(row)):
                    data[j].append(float(row[j]))     
    return data

def DatosVec(path):
    data = [[],[],[]]
    with open(path) as fobj:
        for i, line in enumerate(fobj):
           # data.append([])
            if line.rsplit() != []:
                row = line.rsplit()   
                for j in range(len(row)):
                    data[j].append(float(row[j]))     
    return data

def Kaczmarz(A,N,phi,x0,max_iter,tol):
    x = x0
    r = phi - A.dot(x)
    err = 1
    k = 0
    while k<max_iter and err>tol:
        for i in range(N):
            v = np.eye(1,N,i).reshape(N)
            Av = A @ v
            alpha = np.vdot(Av,r) / np.vdot(Av,Av)
            x += alpha * v
            r = phi - A.dot(x)
        err = np.sqrt(np.real(np.vdot(r,r)))
        if err<tol:
            print("Kaczmarz for D converged in {0} iterations. Error {1}".format(k+1,err))
            return x
        k += 1
    print("Kaczmarz for D did NOT converge in {0} iterations. Error {1}".format(max_iter,err))
    return x      

In [3]:
Nx, Nt = 20, 20
block_x, block_t = 4, 4
x_elements, t_elements = int(Nx/block_x), int(Nt/block_t)
Ntest, Nagg = 15, 2*(block_x*block_t)
Ntot = Nx*Nt
Coords = np.zeros((Nx,Nt,2),dtype=int)
for x in range(Nx):
    for t in range(Nt):
        for s in range(2):
            Coords[x,t,s] = x * Nt * 2 + t * 2 + s
XCoord = np.zeros(2*Nx*Nt,dtype=int)
TCoord = np.zeros(2*Nx*Nt,dtype=int)
SCoord = np.zeros(2*Nx*Nt,dtype=int)
Agg = np.zeros((2*block_x*block_t,x_elements*t_elements),dtype=int)
print("Lattice dimensions Nx={0}, Nt={1}".format(Nx,Nt))
print("Number of aggregates {0}".format(Nagg))
print("Block_x={0}, Block_t={1}".format(block_x,block_t))
print("Number of test vectors {0}".format(Ntest))

Lattice dimensions Nx=20, Nt=20
Number of aggregates 32
Block_x=4, Block_t=4
Number of test vectors 15


In [4]:
#Computing the aggregates
for x in range(block_x):
    for t in range(block_t):
        for s in range(2):
            x0, t0 = x * x_elements, t * t_elements
            x1, t1 = (x+1) * x_elements, (t+1) * t_elements
            aggregate = x * block_t * 2 + t * 2 + s
            count = 0
            for X in range(x0,x1):
                for T in range(t0,t1):
                    i = X * Nt * 2 + T * 2 + s
                    XCoord[i] , TCoord[i], SCoord[i] = X, T, s
                    Agg[aggregate][count] = i
                    count += 1

In [5]:
class AMG():
    def __init__(self,D,nu1,nu2):
        self.test_vectors = np.zeros((Ntest,2*Ntot),dtype=complex) 
        self.P = np.zeros((2*Ntot,Ntest*Nagg),dtype=complex) 
        self.PH = np.zeros((Ntest*Nagg,2*Ntot),dtype=complex) 
        self.D = csc_array(D) #Dirac matrix
        self.Dc = np.zeros((Ntest*Nagg,Ntest*Nagg),dtype=complex) 
        self.nu1 = nu1
        self.nu2 = nu2
        self.it = 0 #Parameter to measure iterations for convergence

    def get_p(self):
        return self.P, self.PH
    def get_Dc(self):
        return self.Dc
    def get_iterations(self):
        return self.it

    def orthonormalize(self):
        v_chopped = [] #Columns of the interpolator
        for i in range(Ntest*Nagg):
            v = np.eye(1,Ntest*Nagg,i).reshape(Ntest*Nagg)
            v_chopped.append(self.P @ v)
        v_chopped = np.array(v_chopped)        
        for i in range(Nagg):
            for nt in range(Ntest):
                for j in range(nt):
                    #vdot(a,b) = a* . b /= b* . a = vdot(b,a)
                    proj = np.vdot(v_chopped[j * Nagg + i],v_chopped[nt * Nagg + i])
                    v_chopped[nt * Nagg + i] -= proj * v_chopped[j * Nagg + i]
                v_chopped[nt*Nagg+i] /= np.linalg.norm(v_chopped[nt*Nagg+i])
        matrix =  np.zeros((2*Ntot,Ntest*Nagg),dtype=complex) 
        for i in range(Ntest*Nagg):
            matrix[:,i] = v_chopped[i]
        self.P = matrix
        self.PH = csc_array(np.conjugate(np.transpose(self.P)))
        self.P = csc_array(self.P)
        self.Dc = (self.PH @(self.D @ self.P)) #matrix multiplication

    def tv(self,Nit):  
        test_vectors_copy = np.zeros((Ntest,2*Ntot),dtype=complex) 
        for i in range(Ntest): 
            for j in range(2*Ntot):
                theta = np.random.uniform(0,2*pi)
                self.test_vectors[i,j] = np.cos(theta) + 1j*np.sin(theta)
        for i in range(Ntest):
            self.test_vectors[i], exit_code = gmres(self.D, self.test_vectors[i], x0=self.test_vectors[i] ,maxiter=20)
        self.P_v() #Assemble P
        self.orthonormalize() #Orthonormalize
        print("Improving interpolator")
        for n in range(Nit):
            print("n={0} out of Nit={1}".format(n,Nit))
            for i in range(Ntest):
                self.test_vectors[i] = self.TwoGrid(1,1e-10,self.test_vectors[i],self.test_vectors[i],False)
            self.P_v() #Assemble P
            self.orthonormalize() #Orthonormalize
            
             
    def P_v(self):
        #Assembling the interpolator
        for can_vec in range(Ntest*Nagg):   
            P_v = np.zeros(2*Ntot,dtype=complex)
            v = np.eye(1,Ntest*Nagg,can_vec).reshape(Ntest*Nagg)
            for j in range(Ntest*Nagg):
                k = int(j / Nagg)
                a = int(j % Nagg)
                for i in range(len(Agg[a])):
                    x, t, s = XCoord[Agg[a,i]], TCoord[Agg[a,i]], SCoord[Agg[a,i]]
                    P_v[Coords[x,t,s]]  += self.test_vectors[k,Coords[x,t,s]] * v[j]
            self.P[:,can_vec] = P_v   
        self.PH = csc_array(np.conjugate(np.transpose(self.P)))
        self.P = csc_array(self.P)
        self.Dc = (self.PH @(self.D @ self.P)) #matrix multiplication
        
    def TwoGrid(self,max_iter,tol,x0,phi,message):
        #Solve D x = phi
        x = x0
        err = 1
        k = 0
        r0 = phi - self.D.dot(x)
        max_err = tol*np.linalg.norm(phi)
        while(k < max_iter and err > max_err):
            if self.nu1 > 0:
                x, exitCode = gmres(self.D, phi, x0=x , rtol=1e-10, atol=0.0, maxiter=nu1) 
            #Coarse grid correction
            Pt_r = self.PH.dot( (phi-self.D.dot(x)))
            Dc_inv, exit_code = bicgstab(self.Dc, Pt_r,x0=Pt_r,maxiter=10000, rtol=1e-10, atol=0.0) #Dc^-1 P^H(phi-Dx)
            if message==True:
                if exit_code == 0:
                    print("Bi-cgstab converged for Dc")
                else:
                    print("Bi-cgstab did NOT converge for Dc")
            x += self.P.dot(Dc_inv)
            if self.nu2>0:
                x, exitCode = gmres(self.D, phi, x0=x ,rtol=1e-10, atol=0.0, maxiter=nu2)
            r = phi - self.D.dot(x)
            err = np.sqrt(np.real(np.vdot(r,r))) #complex dot product
            if message == True:
                print("iteration",k,"error",err)
            if err<=max_err:
                self.it = k+1
                print("two-grid converged in",k+1,"iterations. Error ",err)
                return x
            k+=1
        if message == True:
            print("two-grid did not converge in",max_iter,"iterations. Error ",err)
        self.it = max_iter
        return x

    def print_p(self):
        for i in range(2*Ntot):
            for j in range(Ntest*Nagg):
                print(self.P[i,j], end = " ")
            print('')

In [6]:
dim = Nx*Nt*2
print(dim)
D = np.zeros((dim,dim),dtype=complex)
row, col, re, im = DatosMat("build/D0.dat")
for i in range(dim*dim):
    D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 
phi = np.zeros(dim,dtype=complex)
row, col, re, im = DatosMat("build/Phi0.dat")
for i in range(dim):
    phi[i] = re[i] + 1j*im[i] 

800


In [37]:
#Initial solution
x0 = np.zeros(2*Ntot,dtype=complex)
for i in range(2*Ntot):  
    theta = np.random.uniform(0,2*pi)
    x0[i] = np.cos(theta) + 1j*np.sin(theta)

In [22]:
Dsparse = csc_array(D)
x_gmres = x0
#GMRES
tol = 1e-10
print("---GMRES---")
for i in range(1000):
    x_gmres, exit_code = gmres(Dsparse, phi, x0=x_gmres, rtol=1e-10, atol=0.0, maxiter=1)
    r = phi - Dsparse.dot(x_gmres)
    err = np.sqrt(np.real(np.vdot(r,r)))
    print(i,err,exit_code)
    if err<=tol*np.linalg.norm(phi):
        print("GMRES converged in {0} iterations".format(i+1))
        break
#bi-cgstab
print("---Bi-cgstab---")
x_bi = x0
x_bi, exit_code = bicgstab(Dsparse, phi,x0=x_bi,maxiter=500, rtol=1e-10)
if exit_code == 0:
    print("Converged")
else:
    print("Did not converge")
r = phi - Dsparse.dot(x_bi)
err = np.sqrt(np.real(np.vdot(r,r)))
print("Bi-cgstab error {0}".format(err))

---GMRES---
0 107.2868445094798 1
1 88.59749686907298 1
2 83.78396513630716 1
3 81.07768136242507 1
4 79.14142713109412 1
5 77.82742117554245 1
6 77.399308721317 1
7 77.26306672332338 1
8 77.22616998569441 1
9 77.21116022843356 1
10 77.20361598219806 1
11 77.19934137905561 1
12 77.19701146195003 1
13 77.19577584920154 1
14 77.19514064406079 1
15 77.19481684128601 1
16 77.1946518763996 1
17 77.1945672363435 1
18 77.19452346443558 1
19 77.19450062276185 1
20 77.19448861474568 1
21 77.19448226000065 1
22 77.19447888000157 1
23 77.19447707478044 1
24 77.19447610767412 1
25 77.19447558832474 1
26 77.19447530893495 1
27 77.19447515843142 1
28 77.19447507727692 1
29 77.19447503348411 1
30 77.19447500983955 1
31 77.19447499706816 1
32 77.1944749901677 1
33 77.19447498643852 1
34 77.19447498442281 1
35 77.19447498333317 1
36 77.19447498274407 1
37 77.19447498242556 1
38 77.19447498225334 1
39 77.19447498216022 1
40 77.19447498210987 1
41 77.19447498208264 1
42 77.19447498206792 1
43 77.19447498

In [23]:
x0 = np.zeros(2*Ntot,dtype=complex)
for i in range(2*Ntot):  
    theta = np.random.uniform(0,2*pi)
    x0[i] = np.cos(theta) + 1j*np.sin(theta)
nu1, nu2 = 0, 2
Nit = 3
amg = AMG(D,nu1,nu2)
amg.tv(Nit)
P, PH = amg.get_p()
max_iter = 100
tol = 1e-10
x_sol = amg.TwoGrid(max_iter,tol,x0,phi,True)

Improving interpolator
n=0 out of Nit=3
n=1 out of Nit=3
n=2 out of Nit=3
Bi-cgstab converged for Dc
iteration 0 error 1.8835615107041597
Bi-cgstab converged for Dc
iteration 1 error 0.11544454445335382
Bi-cgstab did NOT converge for Dc
iteration 2 error 0.025472093174758452
Bi-cgstab converged for Dc
iteration 3 error 0.0012887633361801129
Bi-cgstab converged for Dc
iteration 4 error 8.21299355956247e-05
Bi-cgstab converged for Dc
iteration 5 error 6.224595695874749e-06
Bi-cgstab converged for Dc
iteration 6 error 3.0358900007508344e-07
Bi-cgstab did NOT converge for Dc
iteration 7 error 2.3623523167237172e-07
Bi-cgstab converged for Dc
iteration 8 error 9.448215781437055e-09
Bi-cgstab did NOT converge for Dc
iteration 9 error 2.1968099223565486e-08
Bi-cgstab did NOT converge for Dc
iteration 10 error 1.902217531137444e-08
Bi-cgstab did NOT converge for Dc
iteration 11 error 1.323016050090386e-08
Bi-cgstab did NOT converge for Dc
iteration 12 error 2.6191724979044705e-08
Bi-cgstab did

In [16]:
for i in range(2*Ntot):
    line_new = '{:>12} {:>12} {:>12}'.format(x_sol[i], x_bi[i], x_gmres[i])
    print(line_new)

(-27.63362531525767+3.1500414231133402j) (-27.633625313484725+3.150041431243549j) (1.7001872770635518-1.4125988123996722j)
(-25.281573595563362+32.82484895962103j) (-25.281573585276142+32.82484896625063j) (0.0829635611285141-3.1405573915040232j)
(19.739808760545323+0.36266311070307544j) (19.73980876012379+0.3626631053685685j) (-0.15022340124472805+0.6265383699592432j)
(-22.421070371236116-19.654802974417382j) (-22.421070376163858-19.654802967775453j) (1.5155346062747341-0.5706389030434921j)
(-2.2010179926852085-12.091220393337261j) (-2.201017995639991-12.091220392529864j) (1.0708496365233107+0.04859579796150939j)
(-2.9636034997083778+23.469869977309543j) (-2.9636034929666586+23.46986997801647j) (0.3798922724146128-0.8531584103209463j)
(1.8542713587820718-1.2673627988909872j) (1.8542713579954158-1.267362799025669j) (-0.4059379601758149+0.9764054903766511j)
(11.612204308919829-22.41227475668339j) (11.612204302761802-22.412274760160727j) (-0.6882616813666016+0.7822178489815554j)
(-9.85528

### Some tests for a two-grid method using GRMES as smoother

In [31]:
dim = Nx*Nt*2
D = np.zeros((dim,dim),dtype=complex)
phi = np.zeros(dim,dtype=complex)
x0 = np.zeros(2*Ntot,dtype=complex)
iterations = []
for conf in range(50):
    row, col, re, im = DatosMat("build/D{0}.dat".format(conf))
    for i in range(dim*dim):
        D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 
    row, col, re, im = DatosMat("build/Phi{0}.dat".format(conf))
    for i in range(dim):
        phi[i] = re[i] + 1j*im[i] 
    for i in range(2*Ntot):  
        theta = np.random.uniform(0,2*pi)
        x0[i] = np.cos(theta) + 1j*np.sin(theta)
    nu1, nu2 = 0, 2
    Nit = 5
    print("Conf #{0}".format(conf))
    amg = AMG(D,nu1,nu2)
    amg.tv(Nit)
    P, PH = amg.get_p()
    max_iter = 100
    tol = 1e-10
    x_sol = amg.TwoGrid(max_iter,tol,x0,phi,False)
    iterations.append(amg.get_iterations())
iterations = np.array(iterations)
print("Number of iterations: {0} +- {1}".format(np.round(np.mean(iterations),4), np.round(np.std(iterations)/np.sqrt(50),4  ) ))

Conf #0
Improving interpolator
n=0 out of Nit=5
n=1 out of Nit=5
n=2 out of Nit=5
n=3 out of Nit=5
n=4 out of Nit=5
two-grid converged in 5 iterations. Error  2.7641595951113956e-09
Conf #1
Improving interpolator
n=0 out of Nit=5
n=1 out of Nit=5
n=2 out of Nit=5
n=3 out of Nit=5
n=4 out of Nit=5
two-grid converged in 5 iterations. Error  2.7711295537968368e-09
Conf #2
Improving interpolator
n=0 out of Nit=5
n=1 out of Nit=5
n=2 out of Nit=5
n=3 out of Nit=5
n=4 out of Nit=5
two-grid converged in 6 iterations. Error  2.7341030489243244e-09
Conf #3
Improving interpolator
n=0 out of Nit=5
n=1 out of Nit=5
n=2 out of Nit=5
n=3 out of Nit=5
n=4 out of Nit=5
two-grid converged in 5 iterations. Error  2.7871178118567956e-09
Conf #4
Improving interpolator
n=0 out of Nit=5
n=1 out of Nit=5
n=2 out of Nit=5
n=3 out of Nit=5
n=4 out of Nit=5
two-grid converged in 5 iterations. Error  2.678317982547112e-09
Conf #5
Improving interpolator
n=0 out of Nit=5
n=1 out of Nit=5
n=2 out of Nit=5
n=3 out o

### Tests using GMRES

In [27]:
dim = Nx*Nt*2
D = np.zeros((dim,dim),dtype=complex)
phi = np.zeros(dim,dtype=complex)
x0 = np.zeros(2*Ntot,dtype=complex)
tol = 1e-10
iterations = []
for conf in range(8):
    row, col, re, im = DatosMat("build/D{0}.dat".format(conf))
    for i in range(dim*dim):
        D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 
    row, col, re, im = DatosMat("build/Phi{0}.dat".format(conf))
    for i in range(dim):
        phi[i] = re[i] + 1j*im[i] 
    for i in range(2*Ntot):  
        theta = np.random.uniform(0,2*pi)
        x0[i] = np.cos(theta) + 1j*np.sin(theta)
    Dsparse = csc_array(D)
    x_gmres = x0
    #GMRES
    print("Conf #{0}".format(conf))
    max_err = tol*np.linalg.norm(phi)
    err = 1
    k = 0
    while k<1000 and err>max_err:
        x_gmres, exit_code = gmres(Dsparse, phi, x0=x_gmres, rtol=1e-10, atol=0.0, maxiter=1)
        r = phi - Dsparse.dot(x_gmres)
        err = np.sqrt(np.real(np.vdot(r,r)))
        if err<=max_err:
            print("GMRES converged in {0} iterations".format(k+1))
            iterations.append([k+1])
        k += 1
iterations = np.array(iterations)
print("Number of iterations: {0} +- {1}".format(np.round(np.mean(iterations),4), np.round(np.std(iterations)/np.sqrt(50),4  ) ))       

Conf #0
Conf #1
Conf #2
Conf #3
Conf #4
Conf #5
Conf #6
Conf #7
Number of iterations: nan +- nan


### Testing Kaczmarz as a solver

In [41]:
dim = Nx*Nt*2
print(dim)
D = np.zeros((dim,dim),dtype=complex)
row, col, re, im = DatosMat("build/D0.dat")
for i in range(dim*dim):
    D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 
phi = np.zeros(dim,dtype=complex)
row, col, re, im = DatosMat("build/Phi0.dat")
for i in range(dim):
    phi[i] = re[i] + 1j*im[i] 

#Initial solution
x0 = np.zeros(2*Ntot,dtype=complex)
for i in range(2*Ntot):  
    theta = np.random.uniform(0,2*pi)
    x0[i] = np.cos(theta) + 1j*np.sin(theta)

Dsparse = csc_array(D)
tol = 1e-10
max_iter = 5
x = deepcopy(x0)
for i in range(max_iter):
    x = Kaczmarz(D,2*Ntot,phi,x,1,1e-10)
    r = phi - Dsparse.dot(x)
    err = np.sqrt(np.real( np.vdot(r,r) ))
    if err<=tol:
        print("Kaczmarz converged in {0} iterations".format(i+1))
        break
print(x[0])
x = deepcopy(x0)
x=Kaczmarz(D,2*Ntot,phi,x,5,1e-10)
print(x[0])

800
Kaczmarz for D did NOT converge in 1 iterations. Error 28.033085548303088
Kaczmarz for D did NOT converge in 1 iterations. Error 15.70846434498805
Kaczmarz for D did NOT converge in 1 iterations. Error 11.7488055812857
Kaczmarz for D did NOT converge in 1 iterations. Error 9.968897805308629
Kaczmarz for D did NOT converge in 1 iterations. Error 8.912553802483204
(0.2825296053163694+0.00849867573098076j)
Kaczmarz for D did NOT converge in 5 iterations. Error 8.912553802483204
(0.2825296053163694+0.00849867573098076j)
